In [67]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|
IDX002:DIM0016:003|004|
IDX003:DIM0256:005|006|007|008|
IDX004:DIM0016:009|010|
IDX005:DIM0016:011|012|
 Number of tasks:                                         661
 Compute tasks:                                      0.130973 seconds (607.76 k allocations: 59.509 MiB)
 Add results together:                               0.046393 seconds (86.61 k allocations: 16.596 MiB, 37.96% gc time)
 In open_matvec_thread
 Setup threaded jobs:                                0.000166 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 1
 Compute matrix-vector:                              0.015623 seconds (78.87 k allocations: 6.587 MiB)
 Now collect thread results :                        0.000162 seconds (1.06 k allocations: 64.359 KiB)
size(s_bst) = (16278, 1)
size(s_tps) = (49378, 1)


# BST PT2

In [105]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-32, max_iter=100, tol=1e-12, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.002220 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001623 seconds
 Compute <S^2>:                                      0.000447 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 100
 nbody       : 4
 thresh_foi  : 1.0e-32
 max_number  : nothing
 tol         : 1.0e-12
 opt_ref     : true
 verbose     : 1

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ==============================

In [132]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-32)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
v_pt, e_pt = FermiCG.hylleraas_compressed_mp2a(s_bst, v, cluster_ops, clustered_ham, tol=1e-8);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.002995 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001195 seconds
 Compute <S^2>:                                      0.000391 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                      0.125504 seconds (607.76 k allocations: 59.522 MiB)
 Add results together:                               0.026203 seconds (86.61 k allocations: 16.755 MiB)
 Length of input      FOIS:                             16382
 Build exact <X|V|0>:                                0.

In [133]:
e_pt, v_pt = FermiCG.do_fois_pt2(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-64, max_iter=50, tol=1e-8);

 |== Solve for BST PT1 Wavefunction ================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 1.0e-64
 tol         : 1.0e-8
 opt_ref     : true
 verbose     : true

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001835 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001369 seconds
 Compute <S^2>:                                      0.000409 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Diagonalization time:                               0.006073 se

In [112]:
ints.h0

11.795261590783612

# TPSCI PT2

In [18]:
function pt1(ψ, cluster_ops, clustered_ham)
    σ = FermiCG.build_compressed_1st_order_state(ψ, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)
    
end
pt1(ψ, cluster_ops, clustered_ham)

 Number of tasks:                                         661
 Compute tasks:                                      0.077710 seconds (597.90 k allocations: 56.236 MiB)
 Add results together:                             

  0.016173 seconds (85.70 k allocations: 9.271 MiB)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:   485  
 ---------- # Configs    -------------------:  5500  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.027                                   ( 1,1 )( 2,1 )( 1,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.028                                   ( 1,1 )( 0,1 )( 3,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.027                                   ( 2,1 )( 0,1 )( 2,2 )( 1,1 )( 1,1 )
     0.026           4         4             1:2    1